In [1]:
import pandas as pd

df

In [4]:
df = pd.read_csv("data/mview_daten_beschr.csv")
df = df.astype(str)


In [5]:
from transformers import TapasTokenizer, TapasForQuestionAnswering

tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

In [6]:
queries = ["Wie viel FuE Personal war 2011 im Wirtschaftszweig Herstellung von DV-Geräten, elektronischen und optischen Erzeugnissen beschäftigt?", 
           "Wie hoch waren die externen FuE-Aufwendungen im Jahr 2011 im Wirtschaftszweig Metallerzeugung und -bearbeitung?"]

inputs = tokenizer(table=df, queries=queries, padding="max_length", return_tensors="pt")

outputs = model(**inputs)

In [ ]:
logits = outputs.logits.detach()
answers = tokenizer.convert_logits_to_predictions(inputs, logits)

def format_answer(query, answer_coordinates):
    if len(answer_coordinates) == 0:
        return f"The query '{query}' could not be answered."
    
    row, column = answer_coordinates[0]
    if query.startswith("How many"):
        return f"The answer to '{query}' is {df.iloc[row, column]}."
    elif query.startswith("Which class"):
        return f"The answer to '{query}' is {df.iloc[row, 0]}."

formatted_answers = [format_answer(query, coord) for query, coord in zip(queries, answers[0])]

for answer in formatted_answers:
    print(answer)